# HIV drug Experiment
In this notebook we apply our e-CRT algorithm to the HIV data set,
available in [https://hivdb.stanford.edu/dr-summary/comments/PI/](https://hivdb.stanford.edu/dr-summary/comments/PI/).

In conditional independence problems, given data $X\in \mathbb{R}^{n\times d}$ and a response $Y\in\mathbb{R}^n$,
the null hypothesis is:

$H_0: X_j \perp \!\!\! \perp Y \mid X_{-j} $

Where $X_j$ is the j-th feature, and $ X_{-j}$ are all the features except the j-th one.

In our test, a crucial step is to sample the dummy features from the
conditional distribution of $X_{j}\mid X_{-j}$. Yet, in many real-world applications,
this distribution may not be known.
In our code we provide an estimation for the conditional distribution by fitting a multivariate Gaussian,
and demonstrate the performance using this approximation both with [synthetic data](examples/synthetic_experiment.ipynb)
and with [real data](examples/fund_experiment.ipynb).

In this experiment we will examine a case in which the conditional distribution can not be estimated
using a multivariate Gaussian, and demonstrate how the user can provide a function of its own
to sample the dummies.

In [5]:
import pandas as pd
import numpy as np
from data.data import get_hiv_data, get_hiv_clf
from src.e_crt import EcrtTester
from src.sampling_functions import sample_hiv_data
from src.utils import BettingFunction, get_martingale_values

This data set consists of $n=1555$ time steps, each with $d=150$ features.
In this data the features are binary, therefore we sample the dummy features from
a Bernoulli distribution. Since the data size is relatively small, we estimate the
probability of the Bernoulli distribution by training a logistic regression model
on the **entire** data, as we will see next.

In this notebook we test the algorithm on the same $3$ features presented in the paper.

In order to improve robustness, in all our real-data experiments we chose to use $tanh(20 \cdot())$
as our betting function, and to use a batch ensemble with batches $[5, 10, 20]$.

In addition, we use offline learning (using LassoCV) to improve the accuracy of the
learning model.

In [6]:
np.random.seed(0)
j_vec = [10, 61, 62]
batch_list = [5, 10, 20]
n_init = 20
offline=True
g_func = BettingFunction.tanh
sampling_func = sample_hiv_data
save_path = "./results/HIV"
X, Y, features_names = get_hiv_data()

There are 3 input variables that we need to provide in order to change the sampling function
for the dummy features.
1. *sampling_func* -  this function receives the original sample (all the features),
the index of the tested feature, and any additional input you may need in order
to generate the dummies. The function returns a copy of the original sample, with the j-th feature
replaced by the dummy feature.
In our case, this function receives the original sample $X$, the index of
the tested feature $j$, and a classifier that was trained on $X_{-j}$ to predict
$X_j$.
The classifier is passed to the sampling function using a dictionary: *sampling_args*.
2. *sampling_args* - a dictionary with all the information that we wish to pass to
the sampling function, and that is available to us at the **beginning** of the test.
In this experiment we trained the classifier using the entire data set, and therefore
we can pass it using this dictionary. If we trained this classifier online, using
only the available data at each time step, we would use the function *learn_conditional_distribution*.
3. *learn_conditional_distribution* - this function gets the data available at time $t$,
and returns a dictionary of variables to pass to *sampling_func*, along with *sampling_args*.
In this experiment we do not have any learned variables to pass, therefore the function we use
returns an empty dictionary.

In [7]:
results_dict = {
    "idx": j_vec,
    "name": features_names[j_vec],
    "martingale": [],
    "effective n": []
                }

for j in j_vec:
    sampling_args = {"clf": get_hiv_clf(X, j)}
    ecrt_tester = EcrtTester(batch_list=batch_list,
                             j=j,
                             g_func=g_func,
                             n_init=n_init,
                             offline=offline,
                             sampling_func=sampling_func,
                             sampling_args=sampling_args,
                             learn_conditional_distribution=lambda x: {},
                             path=f"{save_path}/feature_{j}")
    rejected = ecrt_tester.run(X, Y)
    martingale, neff = get_martingale_values(ecrt_tester.martingale_dict)
    results_dict["martingale"].append(martingale)
    results_dict["effective n"].append(neff)
    ecrt_tester.save_martingales()

As we presented in the paper, for the first tested feature, named "X.12S", which was not
reported by specialists to affect the drug resistance, the test martingale does not grow.
On the other hand, the test martingale rejects the null for mutation "X.47V", a mutation
that was reported as major, after only 260 time steps used.
Mutation "X.48M" is also labeled as major, and was not classified as such by the test martingale.
Yet, the test martingale reached an e-value of 8.5, and while it is not enough to safely reject
the null, it indicates a substantial evidence against it.

In [8]:
pd.DataFrame(results_dict)

,idx,name,martingale,effective n
0,10,X.12S,1.035570,1555
1,61,X.47V,20.786002,260
2,62,X.48M,8.536322,1555
